# Gaze Estimation

### 1. Generating measureble data 

In [ ]:
meter aqui um demo e depois (como é só demo), mostrar um exemplo de resultados e criar o traindata.txt

In [12]:
#Importing packages:
import cv2 #image manipulation
import sys #manipulating function arguments
import numpy as np #eventual mathemathic operations
from pydarknet import Detector, Image #openpose
import os #operating system interactions
dir_path = os.path.dirname(os.path.realpath(__file__))
from openpose import pyopenpose as op #open pose

ModuleNotFoundError: No module named 'cv2'

In [21]:
# coords2Quadrante calculates to which quadrant (out of 16) a tuple of coordinates refers
# the function's parameters include the coordinates - a tuple - and the image object
# It outputs the quadrant and the image with the calculated quadrant overlined
def coords2Quadrante(coords,img):
    overlay = img.copy() # safe copy of the image to apply alpha
    h, w = img.shape[:2] # gets the image shape (height, width) 
    h = int(h) # converts parameters to integers 
    w= int(w)
    
    # next, it compares the x and the y to a reference from the original image (x/2,x/4 or y/2,y/4)
    # note that the image was devided in 16 rectangles: firstly divided in four and each of these, devided
    # in four again
    
    #____________________________# 0
    # 0  \  1   \   4   \   5    #
    #-----------\----------------# h/4
    # 2  \  3   \   6   \   7    #
    #___________\________________# h/2
    # 8  \  9   \   12  \   13   #
    # ----------\----------------# (3h)/4
    # 10 \  11  \   13  \   15   #
    # __________\________________#  h
    # 0  w/4    w/2    3w/4    w


    if coords[0]<int(w/2):
        if coords[1]<int(h/2):
            if coords[0]<int(w/4):
                if coords[1]<int(h/4):
                    quadrante=0
                    cv2.rectangle(overlay, (0,0), (int(w/4),int(h/4)), (229,88,191), thickness=-1) 
                else:
                    quadrante=2
                    cv2.rectangle(overlay, (0,int(h/4)), (int(w/4),int(h/2)), (229,88,191), thickness=-1)
            else:
                 if coords[1]<int(h/4):
                    quadrante=1
                    cv2.rectangle(overlay, (int(w/4),0), (int(w/2),int(h/4)), (229,88,191), thickness=-1)
                 else:
                    quadrante=3               
                    cv2.rectangle(overlay, (int(w/4),int(h/4)), (int(w/2),int(h/2)), (229,88,191), thickness=-1)
        else:
            if coords[0]<int(w/4): 
                if coords[1]<int((3*h)/4):
                    quadrante=8
                    cv2.rectangle(overlay, (0,int(h/2)), (int(w/4),int((3*h)/4)), (229,88,191), thickness=-1)
                    
                else:
                    quadrante=10
                    cv2.rectangle(overlay, (0,int((3*h)/4)), (int(w/4),int(h)), (229,88,191), thickness=-1)
            else:
                 if coords[1]<int((3*h)/4):
                    quadrante=9
                    cv2.rectangle(overlay, (int(w/4),int(h/2)), (int(w/2),int((3*h)/4)), (229,88,191), thickness=-1)
                 else:
                    quadrante=11               
                    cv2.rectangle(overlay, (int(w/4),int((3*h)/4)), (int(w/2),h), (229,88,191), thickness=-1)
    else:
        if coords[1]<int(h/2): 
            if coords[0]<int((3*w)/4):
                if coords[1]<int(h/4):
                    quadrante=4
                    cv2.rectangle(overlay, (int(w/2),0), (int((3*w)/4),int(h/4)), (229,88,191), thickness=-1)
                 
                else:
                    quadrante=6
                    cv2.rectangle(overlay, ((int(w/2),int(h/4))), (int((3*w)/4),int(h/2)), (229,88,191), thickness=-1)
                   
            else:
                 if coords[1]<int(h/4):
                    quadrante=5
                    cv2.rectangle(overlay, (int((3*w)/4),0), (int(w),int(h/4)), (229,88,191), thickness=-1)
                    
                 else:
                    quadrante=7          
                    cv2.rectangle(overlay, (int((3*w)/4),int(h/4)), (int(w),int(h/2)), (229,88,191), thickness=-1)
                    
        else:
            if coords[0]<int((3*w)/4): 
                if coords[1]<int((3*h)/4):
                    quadrante=12
                    cv2.rectangle(overlay, (int(w/2),int(h/2)), (int((3*w)/4),int((3*h)/4)), (229,88,191), thickness=-1)
                else:
                    quadrante=14
                    cv2.rectangle(overlay, (int(w/2),int((3*h)/4)), (int((3*w)/4),int(h)), (229,88,191), thickness=-1)
            else:
                 if coords[1]<int((3*h)/4):
                    quadrante=13
                 else:
                    quadrante=15               
                    cv2.rectangle(overlay, (int((3*w)/4),int((3*h)/4)), (int(w),int(h)), (229,88,191), thickness=-1)                    
            
    new = cv2.addWeighted(overlay, 0.4, img, 1 - 0.4, 0, img) #purple overlay 
    return quadrante,new #returns the quadrant and a new picture, with the underlined quadrant

In [23]:
coords2Quadrante((924,1234),cv2.imread('image.png'))

NameError: name 'cv2' is not defined

In [14]:
# contextual information - classifying an object in long or short range interaction
def short_long(objeto):
    cena='cena'
    if objeto in ['tv','pottedplant','vase','cat']:
        cena='long'
    if objeto in ['chair','dining table','refrigerator','microwave','sink','bowl','apple','banana','laptop',\
                'keyboard','mouse','knife','fork','backpack']:
        cena='short'
    return cena

In [15]:
short_long('tv')

'long'

### 2. Separating training-set and testing-set 

In [5]:
import random
import shutil
import os

In [6]:
def separaPlus():   
    dir="lixo"
    list = os.listdir(dir)
    j=False
    i=0
    
    for i in range(8):
        file=random.choice(list)
        while j==False:
            if file.endswith('.txt'):
                j=True
            else:
                 file=random.choice(list)
        srcpath0=os.path.join(dir, file)
        shutil.move(srcpath0, "treino")
        shutil.move(srcpath0[:-4]+".png","treino")
        list = os.listdir(dir)
        j=False
        
    for i in range(3):
        file=random.choice(list)
        while j==False:
            if file.endswith('.txt'):
                j=True
            else:
                file=random.choice(list)
        srcpath1=os.path.join(dir, file)
        shutil.move(srcpath1, "teste")
        shutil.move(srcpath1[:-4]+".png","teste")
        list = os.listdir(dir)
        j=False
separaPlus()

FileNotFoundError: [WinError 3] O sistema não conseguiu localizar o caminho especificado: 'lixo'

### 3. Creating the model

In [8]:
from sklearn.svm import SVC 
import numpy as np
import ast
from joblib import dump, load
from sklearn import svm, grid_search
from datetime import datetime

In [9]:
#Opening the traindata file with all the coordinates of the important points.
#The last number of each line is the number of the quadrant 
with open("traindata.txt") as file: #ATTENTION: this filename comes from elsewhere, change if needed
    linestrain = [line.strip() for line in file] #erases the empty spaces in each line from the txt file

    
def coordsandquadrvectors(linestxt)

#Initialization  
numberquad = [] 
Allthecoords = [] 
counter = 0

length = len(linestxt)

#This cicle separates, in each line corresponding to one image
#the coordinates to the Allthecoords vector and the quadrant number to the numberquad vector
while counter != length:
    omega = ast.literal_eval(linestxt[counter]) # ast.literal_eval raises an exception if the input isn't a valid Python datatype, so the code won't be executed if it's not
    Allthecoords += [omega[:10]]
    if omega[-1] == -1: #if the quadrant number is -1 (mistakenly) it is replaced by the quadrant 0
        numberquad +=[0]
    else:
        numberquad +=[omega[-1]]
    counter+=1

return Allthecoords, numberquad, length

CoordsTrain, quadTrain = coordsandquadrvectors(linestrain)
    
#This function below selects the best parameters that will there be used with the SVC
def svc_param_selection(X, y, nfolds):
    Cs = [1000,100] #parameter 1
    gammas = [1e-06] #parameter 2  
    param_grid = {'C': Cs, 'gamma' : gammas} #all in one grid
    grid_search2 = grid_search.GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search2.fit(X, y)
    grid_search2.best_params_
    return grid_search2.best_params_
    
best = svc_param_selection(CoordsTrain, quadTrain, 3) #3 means a 3-fold cross-validation

clf = SVC(C= best['C'],gamma=best['gamma']) #creating the model with the best parameters
clf.fit(CoordsTrain, quadTrain) #fitting the model to the data

now = datetime.datetime.now()
timestamp = str(now.strftime("%Y%m%d_%H-%M-%S")) #the current day and time to have to the classifier name 

classif_name = "classifier_"+timestamp+".joblib" 

dump(clf,classif_name, protocol=2) #creating the classifier file

FileNotFoundError: [Errno 2] No such file or directory: 'treino.txt'

###  4. Testing the model

In [10]:
#ATTENTION: the covariance matrices comments are TEMPORARY since we don't know if it's necessary (one method is implemented but not sure if it's the right one)

#Opening the testdata file with all the coordinates of the important points.
#The last number of each line is the number of the quadrant 
with open("testdata.txt") as file: #ATTENTION: this filename comes from elsewhere, change if needed
    linestest = [line.strip() for line in file]

coordsTest,quadTest, lengthtest = coordsandquadrvectors(linestest)  

#Predicting part: using the classifier to predict 
#(NOT USING THE FILE SAVED, IT ONLY RUNS IF THE CLASSIFIER WAS BUILT IN THE SAME SESSION)

#Extra Initialization 
Qpredicted= []
nmatching = 0 #number of right predictions
confMatrix = np.zeros((16,16))
i=0

while i != lengthtest:
Qpredicted+= [str(clf2.predict(np.array([coordsTest[i]]))).strip('['+']')] #predict
quadTest[i]= str(quadTest[i]) #converting to a string
    
confMatrix[int(Qpredicted[i])][int(quadTest[i])] += 1 
    
if Qpredicted[i] == quadTest[i]:
    nmatching+=1
    
i+=1

persucess = (float(nmatching)/float(lengthtest)) * 100  


print('The percentage of success was: ' + str(persucess)+ ' %')

#print(str(confMatrix))

#def sum1(input):
 #   sum = 0
  #  for row in range (len(input)):
   #     for col in range(len(input[0])):
    #        sum = sum + input[row][col]

#    return sum
#print(sum1(confMatrix))

FileNotFoundError: [Errno 2] No such file or directory: 'treino.txt'

NameError: name 'sum1' is not defined